In [1]:
import os
import sys
import joblib
sys.path.insert(1, '/mnt/d/PowerTAC/Python/python_utils/helper') 

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from statistics import mean
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from read_mongo_collection import HelperToReadMongo
from data_processing import DataProcessor
from network_activities import Network

from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

from tbats import TBATS

sns.set_theme(style="darkgrid")

# See complete data
# pd.set_option('max_columns', None)
# pd.set_option("max_rows", None)
# np.set_printoptions(threshold=np.inf)

In [3]:
helper_mongo = HelperToReadMongo()
data_processor = DataProcessor()
network = Network()

In [4]:
# Specify names of all the databases here
database = 'PowerTAC2021_CUP_Data_Collection'
collection1 = 'DistributionTransaction_and_Report_Info'
collection2 = 'Calendar_Info'
collection3 = 'WeatherReport_Info'

In [5]:
result_path = '/mnt/d/PowerTAC/PowerTAC2021/experiments/forecasting_check/benchmarking_consumption_forecast/'

In [6]:
db_demand = helper_mongo.query_to_mongo(database, collection1, server_ip='192.168.0.106', ssh_username='sanjay', ssh_password='san@9397', remote=False)  
db_cal = helper_mongo.query_to_mongo(database, collection2, server_ip='192.168.0.106', ssh_username='sanjay', ssh_password='san@9397', remote=False)  
db_wthr = helper_mongo.query_to_mongo(database, collection3, server_ip='192.168.0.106', ssh_username='sanjay', ssh_password='san@9397', remote=False)  

db = pd.merge(db_demand, pd.merge(db_wthr, db_cal,  how='outer', left_on=['Game_Name', 'Timeslot'], right_on = ['Game_Name', 'Timeslot']), 
            how='outer', left_on=['Game_Name', 'Timeslot'], right_on = ['Game_Name', 'Timeslot'])

db['Hour_of_Week'] = (db['Day_of_Week']-1)*24 + db['Hour_of_Day']   

In [7]:
insample_df = db[:int(0.8*len(db))]
insample_df = insample_df[['Timeslot', 'Hour_of_Week', 'Temperature', 'Wind_Speed', 'Total_Consumption', 'Total_Production']]

df = db[int(0.8*len(db)):]
df = df[['Timeslot', 'Hour_of_Week', 'Temperature', 'Wind_Speed', 'Total_Consumption', 'Total_Production']]

In [8]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [9]:
def in_sample_naive_mae():
    
    dataframe = insample_df
    original = dataframe['Total_Consumption']
    predicted = dataframe['Total_Consumption'].shift(1, axis=0)

    mae = mean_absolute_error(predicted[1:].values.tolist(), original[1:].values.tolist())
    return mae

In [10]:
df

,Timeslot,Hour_of_Week,Temperature,Wind_Speed,Total_Consumption,Total_Production
17529,1566,102,-3.0,28.0,57227.435284,11623.73
17530,1567,103,-2.0,17.0,72697.549124,18371.59
17531,1568,104,-1.0,28.0,58563.338511,13670.96
17532,1569,105,-2.0,17.0,52240.064457,10463.12
17533,1570,106,-2.0,11.0,52538.979103,16824.83
...,...,...,...,...,...,...
21907,987,3,-2.0,13.0,42469.864976,11578.99
21908,360,96,19.0,7.0,33788.711938,10507.59
21909,361,97,18.0,6.0,35052.789555,4646.36
21910,362,98,18.0,6.0,41695.958646,4738.10


# LSTM Based Prediction

In [11]:
def predict_consumption_usage_lstm(dataframe):
    
    # Configurable Parameters
    look_back = 168
    look_ahead = 24
    num_features = 3

    try:

        features = ['Temperature', 'Wind_Speed', 'Total_Consumption']
        dataframe = dataframe[features]

        model_storage_path = '/mnt/d/PowerTAC/PowerTAC2021/brokers/VidyutVanika21/python/models'
        scaler_storage_path = model_storage_path + '/scalers/net_consumption.save'
        scaler = joblib.load(scaler_storage_path)
        df = data_processor.normalize_minmax(scaler, dataframe, fit = False)

        usage_per_population_index = list(dataframe.columns).index('Total_Consumption')
        cols = list(dataframe.columns)
        cols.remove('Total_Consumption')
        cols.append('Total_Consumption')

        normalized_df = df[cols].values

        test_X, test_Y = data_processor.create_lstm_dataset(normalized_df, num_features = num_features, look_back = look_back, look_ahead = look_ahead)
        model = load_model(model_storage_path + '/Net_Consumption_Demand/best_model.hdf5')
        predicted_value= model.predict(test_X)

        reverse_scaler = data_processor.get_scaler()
        reverse_scaler.min_, reverse_scaler.scale_ = scaler.min_[usage_per_population_index], scaler.scale_[usage_per_population_index]

        predicted_test_Y = np.squeeze(reverse_scaler.inverse_transform(predicted_value))

        original_test_Y = dataframe['Total_Consumption'][look_back:-(look_ahead-1)]
        
        fig = go.Figure([
            go.Scatter(
                name='Prediction',
                y=list(predicted_test_Y[:,0]),
                mode='lines',
            ),
            go.Scatter(
                name='Actual',
                y=original_test_Y.values.tolist(),
                mode='lines',
            )
        ])
        fig.update_layout(
            yaxis_title='Demand (MWh)',
            title='Error Plot',
            hovermode="x"
        )
        fig.write_html(result_path + 'lstm_error_plot.html')

        mase = 0   # Mean Absolute Scaled Error (MAE / MAE_of_in_sample_naive) proposed by Koehler & Hyndman
        mape = 0
        mae = 0
        rmse = 0

        try:
            mae = mean_absolute_error(list(predicted_test_Y[:,0]), original_test_Y.values.tolist())
            mase = mae / in_sample_naive_mae()
            rmse = mean_squared_error(list(predicted_test_Y[:,0]), original_test_Y.values.tolist(), squared=False)
            mape = mean_absolute_percentage_error(predicted_test_Y[:,0], original_test_Y.values.tolist())
        except Exception as e:
            print(e)
            
        print("LSTM_MASE : " + str(mase))
        print("LSTM_MAE : " + str(mae))
        print("LSTM_RMSE : " + str(rmse))
        print("LSTM_MAPE : " + str(mape) + "\n\n")

        f = open(result_path + 'lstm_loss.csv', 'w')
        f.write(str(mase) + ", " + str(mae) + ", " + str(rmse) + ", " + str(mape) + "\n")
        f.close()

    except Exception as e:
        print(e)

# sNaive

In [12]:
def sNaive(dataframe):
    
    original = dataframe['Total_Consumption']
    predicted = dataframe['Total_Consumption'].shift(1, axis=0)
    
    fig = go.Figure([
        go.Scatter(
            name='Prediction',
            y=predicted.values.tolist(),
            mode='lines',
        ),
        go.Scatter(
            name='Actual',
            y=original.values.tolist(),
            mode='lines',
        )
    ])
    fig.update_layout(
        yaxis_title='Demand (MWh)',
        title='Error Plot',
        hovermode="x"
    )
    fig.write_html(result_path + 'sNaive_error_plot.html')

    mase = 0   # Mean Absolute Scaled Error (MAE / MAE_of_in_sample_naive) proposed by Koehler & Hyndman
    mape = 0
    mae = 0
    rmse = 0

    try:
        mae = mean_absolute_error(predicted[1:].values.tolist(), original[1:].values.tolist())
        mase = mae / in_sample_naive_mae()
        rmse = mean_squared_error(predicted[1:].values.tolist(), original[1:].values.tolist(), squared=False)
        mape = mean_absolute_percentage_error(predicted[1:].values, original[1:].values)
    except Exception as e:
        print(e)

    print("sNaive_MASE : " + str(mase))
    print("sNaive_MAE : " + str(mae))
    print("sNaive_RMSE : " + str(rmse))
    print("sNaive_MAPE : " + str(mape) + "\n\n")

    f = open(result_path + 'sNaive_loss.csv', 'w')
    f.write(str(mase) + ", " + str(mae) + ", " + str(rmse) + ", " + str(mape) + "\n")
    f.close()

# ETS (Exponenetial Smoothing)

In [13]:
def ETS(df):
    
    model = ETSModel(df["Total_Consumption"])
    fit = model.fit(maxiter=100)

    # obtained from R
    params_R = [0.99989969, 0.11888177503085334, 0.80000197, 36.46466837, 34.72584983]
    yhat = model.smooth(params_R).fittedvalues

    fig = go.Figure([
        go.Scatter(
            name='Prediction',
            y=fit.fittedvalues,
            mode='lines',
        ),
        go.Scatter(
            name='Actual',
            y=df["Total_Consumption"].values.tolist(),
            mode='lines',
        ),
        go.Scatter(
            name='R fit',
            y=yhat,
            mode='lines',
        )
    ])
    fig.update_layout(
        yaxis_title='Demand (MWh)',
        title='Error Plot',
        hovermode="x"
    )
    fig.write_html(result_path + 'ETS_error_plot.html')

    mase = 0   # Mean Absolute Scaled Error (MAE / MAE_of_in_sample_naive) proposed by Koehler & Hyndman
    mape = 0
    mae = 0
    rmse = 0

    try:
        mae = mean_absolute_error(df["Total_Consumption"].values.tolist(), fit.fittedvalues.tolist())
        mase = mae / in_sample_naive_mae()
        rmse = mean_squared_error(df["Total_Consumption"].values.tolist(), fit.fittedvalues.tolist(), squared=False)
        mape = mean_absolute_percentage_error(df["Total_Consumption"].values, fit.fittedvalues)
    except Exception as e:
        print(e)

    print("ETS_MASE : " + str(mase))
    print("ETS_MAE : " + str(mae))
    print("ETS_RMSE : " + str(rmse))
    print("ETS_MAPE : " + str(mape) + "\n\n")

    f = open(result_path + 'ETS_loss.csv', 'w')
    f.write(str(mase) + ", " + str(mae) + ", " + str(rmse) + ", " + str(mape) + "\n")
    f.close()

    mase = 0   # Mean Absolute Scaled Error (MAE / MAE_of_in_sample_naive) proposed by Koehler & Hyndman
    mape = 0
    mae = 0
    rmse = 0

    try:
        mae = mean_absolute_error(df["Total_Consumption"].values.tolist(), yhat.tolist())
        mase = mae / in_sample_naive_mae()
        rmse = mean_squared_error(df["Total_Consumption"].values.tolist(), yhat.tolist(), squared=False)
        mape = mean_absolute_percentage_error(df["Total_Consumption"].values, yhat)
    except Exception as e:
        print(e)

    print("ETS_R_MASE : " + str(mase))
    print("ETS_R_MAE : " + str(mae))
    print("ETS_R_RMSE : " + str(rmse))
    print("ETS_R_MAPE : " + str(mape) + "\n\n")

    f = open(result_path + 'ETS_R_loss.csv', 'w')
    f.write(str(mase) + ", " + str(mae) + ", " + str(rmse) + ", " + str(mape) + "\n")
    f.close()

# TBATS

In [14]:
def TBATS(df):
    
    estimator = TBATS(seasonal_periods=[7, 365.25])

    # Fit model
    fitted_model = estimator.fit(df["Total_Consumption"])

    # Forecast 14 steps ahead
    y_forecasted = fitted_model.forecast(steps=24)

    fig = go.Figure([
        go.Scatter(
            name='Prediction',
            y=y_forecasted,
            mode='lines',
        ),
        go.Scatter(
            name='Actual',
            y=df["Total_Consumption"].values.tolist(),
            mode='lines',
        )
    ])
    fig.update_layout(
        yaxis_title='Demand (MWh)',
        title='Error Plot',
        hovermode="x"
    )
    fig.write_html(result_path + 'TBATS_error_plot.html')

    mase = 0   # Mean Absolute Scaled Error (MAE / MAE_of_in_sample_naive) proposed by Koehler & Hyndman
    mape = 0
    mae = 0
    rmse = 0

    try:
        mae = mean_absolute_error(df["Total_Consumption"].values.tolist(), y_forecasted.tolist())
        mase = mae / in_sample_naive_mae()
        rmse = mean_squared_error(df["Total_Consumption"].values.tolist(), y_forecasted.tolist(), squared=False)
        mape = mean_absolute_percentage_error(df["Total_Consumption"].values, y_forecasted)
    except Exception as e:
        print(e)

    print("TBATS_MASE : " + str(mase))
    print("TBATS_MAE : " + str(mae))
    print("TBATS_RMSE : " + str(rmse))
    print("TBATS_MAPE : " + str(mape) + "\n\n")

    f = open(result_path + 'TBATS_loss.csv', 'w')
    f.write(str(mase) + ", " + str(mae) + ", " + str(rmse) + ", " + str(mape) + "\n")
    f.close()

In [15]:
predict_consumption_usage_lstm(df)
sNaive(df)
ETS(df)
# TBATS(df)

LSTM_MASE : 1.320001400518347
LSTM_MAE : 8159.554790881423
LSTM_RMSE : 10700.500483126505
LSTM_MAPE : 15.806551119170987


sNaive_MASE : 1.0468580296465313
sNaive_MAE : 6471.126051700213
sNaive_RMSE : 8970.403504533571
sNaive_MAPE : 12.248000985181594




/home/sanjay/anaconda3/envs/powertac_ddpg/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



ETS_MASE : 1.0437792546993996
ETS_MAE : 6452.094683354657
ETS_RMSE : 8764.93074544288
ETS_MAPE : 12.19370522540586


ETS_R_MASE : 1.0487186931194403
ETS_R_MAE : 6482.627695220161
ETS_R_RMSE : 9010.753980952113
ETS_R_MAPE : 12.16722710383911




In [16]:
# from tbats import TBATS
# import numpy as np

# # required on windows for multi-processing,
# # see https://docs.python.org/2/library/multiprocessing.html#windows
# if __name__ == '__main__':
#     np.random.seed(2342)
#     t = np.array(range(0, 160))
#     y = 5 * np.sin(t * 2 * np.pi / 7) + 2 * np.cos(t * 2 * np.pi / 30.5) + \
#         ((t / 20) ** 1.5 + np.random.normal(size=160) * t / 50) + 10
    
#     # Create estimator
#     estimator = TBATS(seasonal_periods=[14, 30.5])
    
#     # Fit model
#     fitted_model = estimator.fit(y)
    
#     # Forecast 14 steps ahead
#     y_forecasted = fitted_model.forecast(steps=14)
    
#     # Summarize fitted model
#     print(fitted_model.summary())